In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("haven")

create_database('blp_dataset')
create_table('products_tmp_haven')

In [ ]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\shop\item")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                desc_content = page_content.find("div", attrs={"class": "column-group half-top-space"})
                
                if desc_content is not None:
                    description = get_content(desc_content).replace("Description ", "")

                ## GET TITLE
                title = ""
                raw_title = page_content.find("title")

                if raw_title is not None:
                    title = get_content(raw_title)
                
                ## GET SELLER
                seller = ""
                raw_seller = page_content.find("a", attrs={"title": "Vendor"})

                if raw_seller is not None:
                    seller = get_content(raw_seller).replace("(0)", "")
                

                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_haven (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                    values = (title, description, "Haven", seller, "t")
                    
                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid
                    
                    ## GET IMAGE
                    raw_name = page_content.find("figure")
                    name = ""
                    
                    if raw_name is not None:
                        raw_name = raw_name.find("a")
                        
                        if raw_name is not None:
                            raw_name = raw_name["href"]
                            
                            name = raw_name.replace("http://havenpghmfqhivfn.onion/", "")
                        
                        if name is not "":
                            url_image = Path(f"{folder_image}\{name}")

                            try:
                                # copy images to new directory, renaming with product id
                                if url_image.exists():
                                    print(f"Image {product_id} exists... creating")

                                    dst_dir = Path(f"D:\images\_haven\{product_id}.jpg")
                                    copyfile(url_image, dst_dir)
                                else:
                                    print(f"Image {product_id} does not exists")
                            except:
                                print(f"Image {product_id} has error. Except")